In [1]:
import os

In [2]:
%pwd

'd:\\py4e\\Machine_Learning_from_scratch\\Machine Learning\\Industry Ready Projects\\Wine-Quality-Prediction-MLOPS-with-MLflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\py4e\\Machine_Learning_from_scratch\\Machine Learning\\Industry Ready Projects\\Wine-Quality-Prediction-MLOPS-with-MLflow'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/yashwanthreddy7178/Wine-Quality-Prediction-MLOPS-with-MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="yashwanthreddy7178"
os.environ["MLFLOW_TRACKING_PASSWORD"]="2f0a09218b79018bb56442916a9544eb88a54c20"

In [8]:
from dataclasses import dataclass
from pathlib import Path
from typing import Dict

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: Dict[str, str]  # Update the type according to the actual content of all_params
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/yashwanthreddy7178/Wine-Quality-Prediction-MLOPS-with-MLflow.mlflow",
           
        )

        return model_evaluation_config


In [10]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        try:
            # Load test data and model
            test_data = pd.read_csv(self.config.test_data_path)
            model = joblib.load(self.config.model_path)

            # Prepare test data
            test_x = test_data.drop([self.config.target_column], axis=1)
            test_y = test_data[[self.config.target_column]]

            # Set MLflow registry URI
            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            mlflow.set_experiment('Wine-Quality-Prediction')

            with mlflow.start_run():
                # Make predictions
                predicted_qualities = model.predict(test_x)

                # Evaluate metrics
                (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

                # Save metrics as local JSON
                scores = {"rmse": rmse, "mae": mae, "r2": r2}
                save_json(path=Path(self.config.metric_file_name), data=scores)
                try:
                    mlflow.log_params(self.config.all_params)
                    print(self.config.all_params)
                    mlflow.log_metric("rmse", rmse)
                    mlflow.log_metric("r2", r2)
                    mlflow.log_metric("mae", mae)

                    if tracking_url_type_store != "file":
                        mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
                    else:
                        mlflow.sklearn.log_model(model, "model")

                    print('Model evaluation and logging into MLflow completed successfully.')

                except mlflow.exceptions.MlflowException as e:
                    print(f'MLflow exception: {e}')
                except Exception as e:
                    print(f'Error: {e}')
                    raise e

        except Exception as e:
            print(f'Error: {e}')
            raise e

In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    print(model_evaluation_config)  # print the configuration values
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-02-05 21:28:00,910: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-05 21:28:00,915: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-05 21:28:00,924: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-05 21:28:00,927: INFO: common: created directory at: artifacts]
[2024-02-05 21:28:00,930: INFO: common: created directory at: artifacts/model_evaluation]
ModelEvaluationConfig(root_dir='artifacts/model_evaluation', test_data_path='artifacts/data_transformation/test.csv', model_path='artifacts/model_trainer/model.joblib', all_params=ConfigBox({'alpha': 0.2, 'l1_ratio': 0.1}), metric_file_name='artifacts/model_evaluation/metrics.json', target_column='quality', mlflow_uri='https://dagshub.com/yashwanthreddy7178/Wine-Quality-Prediction-MLOPS-with-MLflow.mlflow')
[2024-02-05 21:28:02,895: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
{'alpha': 0.2, 'l1_ratio': 0.1}


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2024/02/05 21:28:16 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 5


Model evaluation and logging into MLflow completed successfully.


Created version '5' of model 'ElasticnetModel'.


In [26]:
import mlflow

# Set the MLflow tracking URI and credentials again with the updated version
mlflow.set_tracking_uri('https://dagshub.com/yashwanthreddy7178/Wine-Quality-Prediction-MLOPS-with-MLflow.mlflow')
mlflow.set_experiment('Wine-Quality-Prediction')

# Set environment variables for authentication
import os
os.environ['MLFLOW_TRACKING_USERNAME'] = 'yashwanthreddy7178'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '2f0a09218b79018bb56442916a9544eb88a54c20'

# Start a new MLflow run and log a parameter
with mlflow.start_run():
    mlflow.log_param("test_param", "test_value")
print('Parameter logged successfully with MLflow version 1.28.0.')

Parameter logged successfully with MLflow version 1.28.0.
